In [ ]:
from transformers import T5ForConditionalGeneration
from tqdm import tqdm
from transformers import MarianTokenizer, MarianMTModel, M2M100ForConditionalGeneration, AutoTokenizer
import torch
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained('lcw99/t5-base-korean-paraphrase').to(device)
tokenizer = AutoTokenizer.from_pretrained('lcw99/t5-base-korean-paraphrase')

df_train = pd.read_csv('data/train.csv')
df_general = pd.read_csv('line_general.csv', sep=',', encoding='utf-8')

print("Train data shape:", len(df_train))
print(df_train.head())

print("General data shape:", len(df_general))
print(df_general.head())

df = pd.concat([df_train, df_general], ignore_index=True)
print("Merged data shape:", len(df))

sentence = "7층 방문을 위해 방문록 작성이 필요합니다."
text =  f"paraphrase: {sentence} "

encoding = tokenizer.batch_encode_plus(
            [text],
            max_length=256,
            truncation=True,
            padding="max_length",
            return_tensors="pt",)

source_ids = encoding["input_ids"].to(device, dtype=torch.long)
source_mask = encoding["attention_mask"].to(device, dtype=torch.long)

generated_ids = model.generate(
                input_ids=source_ids,
                attention_mask=source_mask,
                max_length=150,
                num_beams=2,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True)

preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

def paraphrase(text):
    text =  f"paraphrase: {text} "

    encoding = tokenizer.batch_encode_plus(
                [text],
                max_length=256,
                truncation=True,
                padding="max_length",
                return_tensors="pt",)

    source_ids = encoding["input_ids"].to(device, dtype=torch.long)
    source_mask = encoding["attention_mask"].to(device, dtype=torch.long)

    generated_ids = model.generate(
                    input_ids=source_ids,
                    attention_mask=source_mask,
                    max_length=150,
                    num_beams=2,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True)

    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds[0]

def split_nonempty_lines(text):
    return [ln.strip() for ln in str(text).split("\n") if ln.strip()]


# ✅ conversation 하나를 처리하는 함수 (배치 없음)
def reparaphrase_conversation(text):
    lines = split_nonempty_lines(text)

    en_list = []
    ko_list = []

    for line in lines:
        ko_text = paraphrase(line)
        ko_list.append(ko_text)

    # ko_list를 다시 하나의 conv로 합침
    ko_conv = "\n".join(ko_list)

    return  ko_list, ko_conv


# ✅ 전체 데이터 처리
new_rows = []
for i in tqdm(range(len(df)), desc="Back Translating"):
    original_row = df.iloc[i]

    ko_list, ko_conv = reparaphrase_conversation(original_row["conversation"])

    new_rows.append({
        "idx": original_row["idx"],       # 원본 유지
        "class": original_row["class"],   # 원본 유지
        "conversation": ko_conv,          # 변환된 한국어 대화
    })

    if i % 100 == 0:
        print(f"Original Conversation (idx={original_row['idx']}):\n{original_row['conversation']}\n")
        print(f"reparaphrase Conversation:\n{ko_conv}\n")
        print("-" * 50)

new_df = pd.DataFrame(new_rows)

Train data shape: 3950
   idx      class                                       conversation
0    0      협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1    1      협박 대화  길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2    2  기타 괴롭힘 대화  너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3    3      갈취 대화  어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4    4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
General data shape: 800
   idx class                                       conversation
0    0    일반  오늘 날씨 진짜 좋다.\n그치?\n하늘도 맑고.\n응, 어디 놀러 가고 싶어지네.\...
1    1    일반  점심 뭐 먹을래?\n글쎄.\n어제 치킨 먹어서 오늘은 좀 담백한 거.\n그럼 파스타...
2    2    일반  요즘 뭐 재미있는 드라마 있어?\n아, 나 새로 시작한 거 있는데 완전 내 스타일이...
3    3    일반  주말에 뭐 했어?\n친구 만나서 영화 봤어.\n너는?\n나는 집에서 뒹굴뒹굴.\nㅋ...
4    4    일반  퇴근하고 뭐 해?\n바로 집에 가지.\n피곤해서.\n그치, 나도.\n집에 가서 저녁...
Merged data shape: 4750


Back Translating:   0%|          | 1/4750 [00:03<4:11:19,  3.18s/it]

Original Conversation (idx=0):
지금 너 스스로를 죽여달라고 애원하는 것인가?
 아닙니다. 죄송합니다.
 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.
 정말 잘못했습니다.
 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.
 죄송합니다. 정말 잘못했습니다.
 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.
 선택 못하겠습니다. 한번만 도와주세요.
 그냥 다 죽여버려야겠군. 이의 없지?
 제발 도와주세요.

reparaphrase Conversation:
지금 너는 스스로를 죽여달라고 하는 것인가?
아니요 제가 잘못한 게 없습니다.
죽을 거면 혼자 죽고 우리까지 죽게 해?
정말 잘못한 게 틀림없는데도 변명을 왜 안 해?
니가 선택할 수 있는 것은 네 가족을 죽이는 것이 아니야.
정말 잘못한 게 너무 많아서 말도 못 건넸어요.
선택권이 너한테 있어. 만약 네가 선택하지 못하면 모든 가족이 너의 손에 걸려 죽을 거야.
한 번만 더 기회를 주시면 선택할 수 있게 해 보겠습니다.
그렇다면 전부 다 때려 부수어야 하는 거네?
제가 힘을 좀 보태 달라 부탁하고 싶은 것이 있습니다.

--------------------------------------------------


Back Translating:   0%|          | 15/4750 [00:49<4:13:30,  3.21s/it]

In [ ]:
new_df.to_csv('data/reparaphrased_data.csv', index=False)